In [1]:
import os
import re
import numpy as np
import pandas as pd
import ssl

#Base URL for example
url = 'https://www.ssa.gov/oact/babynames/decades/names1950s.html'

#Generator for dynamic URLs
url_list = []
for year in list(map(str, np.arange(1940,2020, 10))):
    new_url = 'https://www.ssa.gov/oact/babynames/decades/names'+year+'s.html'
    url_list.append(new_url)
    
#Insurance for web scraper
ssl.create_default_https_context = ssl._create_unverified_context

#Scrape name tables from website
table_list = []
for my_url in url_list:
    new_table = pd.read_html(my_url)
    table_list.append(new_table)
    
#Check table schema
#for table in range(0,8):
    #print(table_list[table][0].columns.tolist()) #DEBUG
    
#Create DataFrame and concatenate tables together
new_names = pd.DataFrame(None)

for table in range(0, len(table_list)):
    new_names = pd.concat([new_names, table_list[table][0].iloc[:(len(table_list[table][0])-1), [1,3]]], axis=0, ignore_index=True)

#len(new_names) #DEBUG

#Pull all names into one list and add sexes
new_names.columns = ['M', 'F']
df_male = pd.DataFrame(set(new_names.M), columns=['First_Name'])
df_male['Sex'] = 'M'
df_female = pd.DataFrame(set(new_names.F), columns=['First_Name'])
df_female['Sex'] = 'F'
df_first_names = pd.concat([df_male, df_female], axis=0) #Add name dfs togethers
df_first_names.reset_index(inplace=True, drop=True) #Reset index to include total number of names

#Collect surnames from the internet
surname_url = 'https://en.wikipedia.org/wiki/List_of_most_common_surnames_in_North_America'

surname_tables = pd.read_html(surname_url)

#Pull surnames into one DataFrame
df_surnames = pd.DataFrame(None)

for t in surname_tables:
    if 'Name' in t.columns.tolist():
        df_surnames = pd.concat([df_surnames, t.loc[:, ['Name']]], axis=0)
    elif  'Surname' in t.columns.tolist():
        df_surnames = pd.concat([df_surnames, t.loc[:, ['Surname']]], axis=0)
    else:
        pass
    
df_surnames.reset_index(inplace=True, drop=True) #Reset index

#Collapse columns
new_surnames = []
for idx in df_surnames.index:
    for col in df_surnames.columns:
        if type(df_surnames.loc[idx, col]) == str:
            new_surnames.append(df_surnames.loc[idx, col])
        else:
            pass
        
df_last_names = pd.DataFrame(new_surnames, columns=['Last_Name'])

#Output files to CSV
#df_first_names.to_csv('First_Names.csv', index=False, sep='\t')
#df_last_names.to_csv('Last_Names.csv', index=False, sep='\t')

['https://www.ssa.gov/oact/babynames/decades/names1940s.html',
 'https://www.ssa.gov/oact/babynames/decades/names1950s.html',
 'https://www.ssa.gov/oact/babynames/decades/names1960s.html',
 'https://www.ssa.gov/oact/babynames/decades/names1970s.html',
 'https://www.ssa.gov/oact/babynames/decades/names1980s.html',
 'https://www.ssa.gov/oact/babynames/decades/names1990s.html',
 'https://www.ssa.gov/oact/babynames/decades/names2000s.html',
 'https://www.ssa.gov/oact/babynames/decades/names2010s.html']